In [42]:
#!pip3 install turicreate
#conda install gensim worked

In [50]:
import re
import gensim
from gensim import corpora, models, similarities
import nltk
import pandas as pd
import os
import codecs
import gensim.models.word2vec as w2v
import time

In [64]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
# to use multiple cores for heavy duty functions, but the installation did not work
# import turicreate

In [65]:
# # word2vec with 50 embeddings
# def getWord2Vec(toFeed, dim=50):
#     return gensim.models.Word2Vec(toFeed, min_count=1,  size=dim)

In [4]:
corpus = pd.read_csv("/home/ubuntu/datasets/cancer-clinical-trials-eligibility/labeledEligibilitySample1000000.csv", 
                     sep='\t',
                     error_bad_lines=False,
                     header=None)

In [5]:
len(corpus)

1000000

In [6]:
corpus.head()

,0,1
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [9]:
corpus.iloc[:,0].unique()

array(['__label__0', '__label__1'], dtype=object)

In [11]:
corpus.iloc[:,1].head()

0    study interventions are recombinant CD40-ligan...
1    study interventions are Liposomal doxorubicin ...
2    study interventions are BI 836909 . multiple m...
3    study interventions are Immunoglobulins . recu...
4    study interventions are Paclitaxel . stage ova...
Name: 1, dtype: object

In [12]:
corpus_data= pd.DataFrame(corpus.iloc[:,1])

In [13]:
corpus.shape, type(corpus)

((1000000, 2), pandas.core.frame.DataFrame)

In [14]:
corpus.columns = ['label','text']
corpus.head()

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [15]:
corpus_blob = corpus.groupby('label')['text'].apply(' '.join).reset_index()
corpus_blob

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__1,study interventions are Prednisone . patients ...


In [16]:
inclusion_criteria_0 = corpus_blob['text'][0]
len(inclusion_criteria_0),inclusion_criteria_0[0:500]

(83466072,
 'study interventions are recombinant CD40-ligand . melanoma skin diagnosis and no active cns metastases by ct scan or mri study interventions are Liposomal doxorubicin . colorectal cancer diagnosis and cardiovascular study interventions are BI 836909 . multiple myeloma diagnosis and indwelling central venous cateder or willingness to undergo intra venous central line placement study interventions are Immunoglobulins . recurrent fallopian tube carcinoma diagnosis and patients are allowed to receiv')

In [17]:
inclusion_criteria = corpus['text']

In [32]:
stoplist = set('for a of the and to in is at an must be with are but not no none has have other from as prior or except none see below . study , use " one two three four five six patients before start greater than any allowed by for they since'.split())
texts = [[word for word in inclusion_criteria.split() if word not in stoplist]
         for inclusion_criteria in inclusion_criteria]

In [33]:
len(texts)

1000000

In [34]:
texts[11:20]

[['interventions',
  'Androgens',
  'prostate',
  'cancer',
  'diagnosis',
  'patient',
  'characteristics'],
 ['interventions',
  'Bortezomib',
  'multiple',
  'myeloma',
  'diagnosis',
  'renal',
  'insufficiency',
  's'],
 ['interventions',
  'Prednisone',
  'testicular',
  'lymphoma',
  'diagnosis',
  'bulky',
  'disease',
  'greater_than',
  'equal_than',
  'cm',
  'least',
  'dimension'],
 ['interventions',
  'Razoxane',
  'unspecified',
  'adult',
  'solid',
  'tumor',
  'protocol',
  'specific',
  'diagnosis',
  'platelet',
  'count',
  'greater_than',
  'hundred',
  'zero',
  'mm³'],
 ['interventions',
  'Fludarabine',
  'natural_killer',
  'cell',
  'lymphoblastic',
  'leukemia',
  'lymphoma',
  'diagnosis',
  'bilirubin',
  'more',
  'doc',
  'mg',
  'dl',
  'alanine',
  'aminotransferase',
  'alt',
  'aspartate',
  'aminotransferase',
  'ast',
  'alkaline',
  'phosphatase',
  'alp',
  'more',
  'times',
  'upper',
  'limit',
  'normal',
  'uln'],
 ['interventions',
  'Fluda

In [35]:
len(texts), texts[1:10]

(1000000,
 [['interventions',
   'Liposomal',
   'doxorubicin',
   'colorectal',
   'cancer',
   'diagnosis',
   'cardiovascular'],
  ['interventions',
   'BI',
   '836909',
   'multiple',
   'myeloma',
   'diagnosis',
   'indwelling',
   'central',
   'venous',
   'cateder',
   'willingness',
   'undergo',
   'intra',
   'venous',
   'central',
   'line',
   'placement'],
  ['interventions',
   'Immunoglobulins',
   'recurrent',
   'fallopian',
   'tube',
   'carcinoma',
   'diagnosis',
   'receive',
   'required',
   'receive',
   'additional',
   'cytotoxic',
   'regimens',
   'management',
   'recurrent',
   'persistent',
   'disease',
   'more',
   'non',
   'platinum',
   'non',
   'taxane',
   'regimen'],
  ['interventions',
   'Paclitaxel',
   'stage',
   'ovarian',
   'cancer',
   'diagnosis',
   'recovered',
   'effects',
   'recent',
   'surgery',
   'radiotherapy',
   'therapy'],
  ['interventions',
   'Antibodies,',
   'Monoclonal',
   'recurrent',
   'verrucous',
   'carc

In [28]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
type(texts)

list

In [36]:
#f = codecs.open('C:\Python26\text.txt', 'r', 'utf-8-sig')
#text = f.read()
tok_corp = nltk.word_tokenize(str(texts))

In [37]:
len(tok_corp), tok_corp[1:20]

(49526478,
 ['[',
  "'interventions",
  "'",
  ',',
  "'recombinant",
  "'",
  ',',
  "'CD40-ligand",
  "'",
  ',',
  "'melanoma",
  "'",
  ',',
  "'skin",
  "'",
  ',',
  "'diagnosis",
  "'",
  ','])

### Let's start with no specific input for worker

In [53]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 58.950172424316406


### Let's start with 1 worker 

In [55]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    workers=1,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 102.41815304756165


### Let's try with 3 workers

In [51]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    workers=3,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 59.04199290275574


### Let's try with 7 workers

In [56]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    workers=7,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 45.07705521583557


### Now let's try with 16 workers

In [52]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    workers=16,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 44.11270618438721


# Let's move on to inferences

In [67]:
print(ct2vec['aml'])

[ 2.4131155   0.24132387  0.43815342  0.36739308 -0.83589536  2.0517
  0.01133581  0.02461856  1.8907418   1.0808561  -1.0471681   3.2787938
  0.06309839  0.24490382  1.7549249   0.41513923  1.3437625   3.062722
 -2.1651413   0.12668914 -6.3054514  -2.5788245  -4.08834     2.128924
  3.7871323  -0.8519633   0.1281437  -0.53177387  0.7217792  -1.0318798
 -6.516103    0.5657095  -0.85469854 -0.8715294  -5.5852885   1.7638673
 -1.4840723   2.7711775  -2.9290268   1.7543273  -2.1623347   0.40513882
  3.5259671  -1.3860146  -1.7506224  -3.2872171  -1.9815975  -0.23040795
  0.10585112 -0.21160589  1.2898772   3.8753004  -3.170181   -2.271826
  2.5881205   0.06941617  2.4410079  -1.1882691  -1.4426079  -0.64798254
  0.07357289  2.1005      1.0163097  -1.7964715  -0.07074342 -1.8445625
  0.5680017  -1.8165491  -0.23063196  0.70163256 -0.34674612  1.4038872
  2.4477835   0.14748792  0.49375564  0.09416357  1.319798    0.8012989
  1.232327    0.8720661   0.24465881  1.7936103   0.2271051   1.664

^ This is how the actual embeddings look like for a word. Any concept is represented using a collection of 100 variables as we have specified it in word2vec function call above.

In [68]:
len(ct2vec['aml'])

100

In [69]:
ct2vec.most_similar('aml')

[('mds', 0.6901994347572327),
 ('myelodysplasia', 0.6220169067382812),
 ('crthree', 0.5785877704620361),
 ('cml', 0.5752544403076172),
 ('mpn', 0.5641536116600037),
 ('raeb', 0.5424069166183472),
 ('apl', 0.5397809147834778),
 ('rcmd', 0.5383357405662537),
 ('blast', 0.5380486249923706),
 ('lthree', 0.5368593335151672)]

^ This is interesting as MDS is precursor to AML, MDS trial is often combined with AML.

In [70]:
ct2vec.most_similar('Sorafenib')

[('Niacinamide', 0.9907951354980469),
 ('Sunitinib', 0.9198187589645386),
 ('Bevacizumab', 0.8942933082580566),
 ('Tremelimumab', 0.8942492008209229),
 ('Everolimus', 0.8913912177085876),
 ('Sirolimus', 0.8866170644760132),
 ('Interferons', 0.8779564499855042),
 ('Trebananib', 0.8662943840026855),
 ('Interferon-alpha', 0.8598595261573792),
 ('Epothilones', 0.858574390411377)]

Let's try the blockbuster drug Keytruda

In [71]:
ct2vec.most_similar('Pembrolizumab')

[('Nivolumab', 0.9329681992530823),
 ('Sirolimus', 0.8790919780731201),
 ('Everolimus', 0.8767455220222473),
 ('Antibodies', 0.8755160570144653),
 ('Vaccines', 0.8712477087974548),
 ('Azacitidine', 0.86952805519104),
 ('Carboplatin', 0.8693208694458008),
 ('Immunoglobulins', 0.8668407797813416),
 ('Bevacizumab', 0.8646867275238037),
 ('Tremelimumab', 0.8559767007827759)]

Pembrolizumab is a CheckPoint Inhibitor which targets PD-1/PD-L1. Interestingly we can see a few more CPIs in the results like Nivolumab, Tremelimumab. Additional, these CPIs are often combined with mAb like Bevacizumab or chemo like Carboplatin

Let's try Enzalutamide a drug used in Castrate-Resistant Prostate Cancer.

In [72]:
ct2vec.most_similar('Enzalutamide')

[('Androgens', 0.8064804077148438),
 ('Atrasentan', 0.8015974760055542),
 ('Deslorelin', 0.7961522340774536),
 ('Hormones', 0.794691801071167),
 ('Methyltestosterone', 0.791942834854126),
 ('Antagonists', 0.7792664766311646),
 ('Denosumab', 0.7791566848754883),
 ('Goserelin', 0.7789286971092224),
 ('Leuprolide', 0.7720438241958618),
 ('Diphosphonates', 0.7693747878074646)]

These results are not that useful. Enzalutamide is a second generation anti-androgen drug (NAAD). I was expecting something like Apalutamide or Abiraterone.

In [73]:
ct2vec.most_similar('Estrogen')

[('Modulators', 0.9232880473136902),
 ('Receptor', 0.9219001531600952),
 ('Selective', 0.8934230804443359),
 ('Androgen', 0.7948561310768127),
 ('Antagonists', 0.7755141258239746),
 ('Sleepiness', 0.7606209516525269),
 ('Leukotriene', 0.7423794269561768),
 ('Female', 0.7398806214332581),
 ('Esterified', 0.7373208999633789),
 ('TNBC', 0.7357364296913147)]

^ This is again a moderately useful result. Estrogen is one of the hormone and Estrogen receptor plays a key role in a type of Breast Cancer, the other 2 being Progesteron and HER2 (both missing). When all three have negative status we call it a Triple-Negative Breast Cancer, which you can see above as TNBC. 